<a href="https://colab.research.google.com/github/adiack/ElementsMVC_dotnetcore_Demo/blob/master/Finetune_Dance_Diffusion_JP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dance Diffusion finetune

https://twitter.com/2feet6inches

事前にモデルのダウンロード&Googleドライブへの格納が必要

参考: https://colab.research.google.com/github/Harmonai-org/sample-generator/blob/main/Dance_Diffusion.ipynb

# セットアップ

In [ ]:
#@title GPUステータスのチェック
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
    #!nvidia-smi
    nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
else:
    #!nvidia-smi -i 0 -e 0
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
    nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_ecc_note)

GPU 0: Tesla T4 (UUID: GPU-3d1c09c8-dade-4fdf-f889-99d9affee983)



In [ ]:
#@title Google Driveのマウント
import subprocess, os, sys, ipykernel

def gitclone(url, targetdir=None):
    if targetdir:
        res = subprocess.run(['git', 'clone', url, targetdir], stdout=subprocess.PIPE).stdout.decode('utf-8')
    else:
        res = subprocess.run(['git', 'clone', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def pipi(modulestr):
    res = subprocess.run(['pip', 'install', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def pipie(modulestr):
    res = subprocess.run(['git', 'install', '-e', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def wget(url, outputdir):
    res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

try:
    from google.colab import drive
    print("Google Colab detected. Using Google Drive.")
    is_colab = True
    google_drive = True #@param {type:"boolean"}
    #@markdown Click here if you'd like to save the diffusion model checkpoint file to (and/or load from) your Google Drive:
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")

if is_colab:
    if google_drive is True:
        drive.mount('/content/drive')
        root_path = '/content/drive/MyDrive/AI/Bass_Diffusion'
    else:
        root_path = '/content'
else:
    root_path = os.getcwd()

import os
def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)

initDirPath = f'{root_path}/init_audio'
createPath(initDirPath)
outDirPath = f'{root_path}/audio_out'
createPath(outDirPath)

if is_colab:
    if google_drive and not save_models_to_google_drive or not google_drive:
        model_path = '/content/models'
        createPath(model_path)
    if google_drive and save_models_to_google_drive:
        model_path = f'{root_path}/models'
        createPath(model_path)
else:
    model_path = f'{root_path}/models'
    createPath(model_path)

Google Colab detected. Using Google Drive.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Install dependencies
!git clone https://github.com/harmonai-org/sample-generator
!pip install /content/sample-generator

Cloning into 'sample-generator'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 203 (delta 7), reused 1 (delta 0), pack-reused 185
Receiving objects: 100% (203/203), 59.46 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (113/113), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/sample-generator
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Created wheel for sample-generator: filename=sample_generator-1.0.

以下を実行する前にGoogleドライブに以下のディレクトリを作成し、トレーニング用の音声ファイル(wav)を格納しておくこと
```
/マイドライブ/AI/input/
```

In [ ]:
#@title トレーニングデータを1秒ごとのチャンクに分割する

!pip install git+https://github.com/drscotthawley/aeiou.git
from aeiou import chunkadelic as ca 
import os
from IPython.display import clear_output

class ar:
  input_paths = ""
  output_path = ""
  workers = os.cpu_count() + 4
  nomix = True
  chunk_size = 65535
  sr = 48000
  overlap= 0.5 
  strip=False 
  thresh=-70 

args = ar()

#@markdown  stem of new filename(s) to be output as chunks
new_filename = "trim"    #@param {type:"string"}
args.new_filename = new_filename

#@markdown   how big each audio chunk is, in samples
chunk_size =      131072 #@param {type:"integer"}
args.chunk_size = chunk_size

#@markdown audio sample rate in Hz 
sr =             48000 #@param {type:"integer"}
args.sr = sr

#@markdown  fraction of each chunk to overlap between hops
overlap= 0.5    #@param {type:"number"}
args.overlap = overlap

#@markdown strip silence: chunks with max power in dB below this value will not be saved to files
strip=True    #@param {type:"boolean"}
args.strip = strip

#@markdown threshold in dB for determining what counts as silence 
thresh=-40 #@param {type:"number"}  
args.thresh = thresh  

#@markdown THIS IS WHERE YOU SPECIFY THE FOLDER THAT CONTAINS THE AUDIO FILE TO PROCESS
#workingpath = "/content/drive/MyDrive/TRAIN/chunkme/" #@param {type:"string"}
workingpath = "/content/drive/MyDrive/AI/input/" #@param {type:"string"}


#@markdown THIS IS WHERE YOU SPECIFY THE FOLDER TO OUTPUT THE CHUNKED OUTPUT FILES INTO
chunked_folder_name="chunked" #@param {type:"string"}

out_path = workingpath+chunked_folder_name
#out_path = '/kaggle/temp/'

args.output_path =  out_path
args.input_paths =  workingpath
args.new_filename = new_filename

if not os.path.exists(out_path):
    os.makedirs(out_path)

filelist = os.listdir(workingpath)
print(filelist)

filenames = ca.get_audio_filenames(args.input_paths) 
n = len(filenames)   
print(f"  Got {n} input filenames") 

print("Processing files (in parallel)")
wrapper = ca.partial(ca.process_one_file, filenames, args)
r = ca.process_map(wrapper, range(0, n), chunksize=1, max_workers=args.workers)  # different chunksize used by tqdm. max_workers is to avoid annoying other ppl

clear_output()
print("DOME")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/drscotthawley/aeiou.git to /tmp/pip-req-build-ga7lj6ci
  Running command git clone -q https://github.com/drscotthawley/aeiou.git /tmp/pip-req-build-ga7lj6ci
     |████████████████████████████████| 143 kB 30.7 MB/s 
     |████████████████████████████████| 1.6 MB 56.2 MB/s 
  Created wheel for aeiou: filename=aeiou-0.0.11-py3-none-any.whl size=23627 sha256=efb67f4afa1675916c0a95585f321861026648c4c2acaf2b5f30d576483f5754
  Stored in directory: /tmp/pip-ephem-wheel-cache-bg46tksu/wheels/ab/8a/d7/7cfe12f88dd2a48337ffa6d5e0965841274b478a549c5e207e
Successfully built aeiou
['lofi.wav', 'chunked']
  Got 1 input filenames
Processing files (in parallel)


  0%|          | 0/1 [00:00<?, ?it/s]

Resampling /content/drive/MyDrive/AI/input/lofi.wav from 44100 Hz to 48000 Hz
skipping chunk /content/drive/MyDrive/AI/input/chunked/contentdriveMyDriveAIinputlofi--459.wav because it's 'silent' (below threhold of -40 dB).
skipping chunk /content/drive/MyDrive/AI/input/chunked/contentdriveMyDriveAIinputlofi--460.wav because it's 'silent' (below threhold of -40 dB).
skipping chunk /content/drive/MyDrive/AI/input/chunked/contentdriveMyDriveAIinputlofi--1292.wav because it's 'silent' (below threhold of -40 dB).
skipping chunk /content/drive/MyDrive/AI/input/chunked/contentdriveMyDriveAIinputlofi--1394.wav because it's 'silent' (below threhold of -40 dB).
skipping chunk /content/drive/MyDrive/AI/input/chunked/contentdriveMyDriveAIinputlofi--1585.wav because it's 'silent' (below threhold of -40 dB).
skipping chunk /content/drive/MyDrive/AI/input/chunked/contentdriveMyDriveAIinputlofi--1586.wav because it's 'silent' (below threhold of -40 dB).
skipping chunk /content/drive/MyDrive/AI/input/c

NameError: ignored

# トレーニング

In [ ]:
#@markdown Log in to [Weights & Biases](https://wandb.ai/) for run tracking
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
#@markdown Name for the finetune project, used as the W&B project name, as well as the directory for the saved checkpoints
NAME="lofi-finetune" #@param {type:"string"}

#@markdown Path to the directory of audio data to use for fine-tuning
TRAINING_DIR="/content/drive/MyDrive/AI/input/chunked" #@param {type:"string"}

#@markdown Path to the checkpoint to fine-tune
CKPT_PATH="/content/drive/MyDrive/AI/models/gwf-440k.ckpt" #@param {type:"string"}

#@markdown Directory path for saving the fine-tuned outputs
OUTPUT_DIR="/content/drive/MyDrive/AI/models/DanceDiffusion/finetune" #@param {type:"string"}

#@markdown Number of training steps between demos
DEMO_EVERY=250 #@param {type:"number"}

#@markdown Number of training steps between saving model checkpoints
CHECKPOINT_EVERY=500 #@param {type:"number"}

#@markdown Sample rate to train at
SAMPLE_RATE=48000 #@param {type:"number"}

#@markdown Number of audio samples per training sample
SAMPLE_SIZE=65536 #@param {type:"number"}

#@markdown If true, the audio samples provided will be randomly cropped to SAMPLE_SIZE samples
#@markdown
#@markdown Turn off if you want to ensure the training data always starts at the beginning of the audio files (good for things like drum one-shots)
RANDOM_CROP=True #@param {type:"boolean"}

#@markdown Batch size to fine-tune (make it as high as it can go for your GPU)
BATCH_SIZE=2 #@param {type:"number"}

#@markdown Accumulate gradients over n batches, useful for training on one GPU. 
#@markdown
#@markdown Effective batch size is BATCH_SIZE * ACCUM_BATCHES.
#@markdown
#@markdown Also increases the time between demos and saved checkpoints
ACCUM_BATCHES=4 #@param {type:"number"}

random_crop_str = f"--random-crop True" if RANDOM_CROP else ""

# Escape spaces in paths
CKPT_PATH = CKPT_PATH.replace(f" ", f"\ ")
OUTPUT_DIR = f"{OUTPUT_DIR}/{NAME}".replace(f" ", f"\ ")

%cd /content/sample-generator/

!python3 /content/sample-generator/train_uncond.py --ckpt-path $CKPT_PATH --name $NAME --training-dir $TRAINING_DIR --sample-size $SAMPLE_SIZE --sample-rate $SAMPLE_RATE --batch-size $BATCH_SIZE --demo-every $DEMO_EVERY --demo-steps 100 --checkpoint-every $CHECKPOINT_EVERY --num-workers 2 $random_crop_str --save-path $OUTPUT_DIR

/content/sample-generator
Using device: cuda
/content/traning
Random crop: True
wandb: Currently logged in as: 2f6i. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.3
wandb: Run data is saved locally in ./wandb/run-20220929_135903-38fi2naq
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run comic-field-2
wandb: ⭐️ View project at https://wandb.ai/2f6i/ff-pre-finetune
wandb: 🚀 View run at https://wandb.ai/2f6i/ff-pre-finetune/runs/38fi2naq
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), us

In [ ]:
!ls -lah /content/drive/MyDrive/AI/input

total 935M
-rw------- 1 root root 935M Sep 29 15:03 lofi.wav
